# Week 2 - Lab Session 3

In this lab session we are going to look at how to calculate confidence intervals and run a t-test using Python. 

As per last week, you should start by running the existing cells which will load in some libraries and a required dataset. Note that these cells may throw a 'FutureWarning' which can be safely ignored. 

In [1]:
import statsmodels.api as sm
import pandas as pd
import scipy as sp
import numpy as np

In [2]:
df = sm.datasets.cpunish.load_pandas().data

In [3]:
df.describe()

,EXECUTIONS,INCOME,PERPOVERTY,PERBLACK,VC100k96,SOUTH,DEGREE
count,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000
mean,4.352941,35462.705882,13.494118,14.258824,638.764706,0.411765,0.221765
std,8.695841,4939.735491,3.456781,9.793943,233.578555,0.507300,0.045721
min,1.000000,26954.000000,8.200000,1.800000,321.000000,0.000000,0.160000
25%,1.000000,32108.000000,10.600000,7.200000,463.000000,0.000000,0.190000
50%,1.000000,34743.000000,13.100000,12.200000,591.000000,0.000000,0.210000
75%,3.000000,37421.000000,16.400000,20.000000,886.000000,1.000000,0.250000
max,37.000000,45844.000000,18.800000,32.100000,1051.000000,1.000000,0.310000


In [4]:
df.head()

,EXECUTIONS,INCOME,PERPOVERTY,PERBLACK,VC100k96,SOUTH,DEGREE
0,37.0,34453.0,16.7,12.2,644.0,1.0,0.16
1,9.0,41534.0,12.5,20.0,351.0,1.0,0.27
2,6.0,35802.0,10.6,11.2,591.0,0.0,0.21
3,4.0,26954.0,18.4,16.1,524.0,1.0,0.16
4,3.0,31468.0,14.8,25.9,565.0,1.0,0.19


Let's first get a mean and confidence interval for the income variable in the dataset. We can do that with the following set of commands:

In [5]:
mean = df['INCOME'].mean()
se = df['INCOME'].std()/np.sqrt(df['INCOME'].shape[0])
sp.stats.norm.interval(0.95, loc=mean, scale=se)

(33114.547834868215, 37810.86392983767)

Check that you understand how this is calculated! Write some code which approximately duplicates the numbers above. Recall the formula for an approximate 95% confidence interval:

\begin{equation*}
95\%\ CI\ =\ mean\ ±\ (1.96\ *\ standard\ error)
\end{equation*}

In [6]:
(mean - (1.96 * se), mean + (1.96 * se))

(33114.50468612019, 37810.9070785857)

The amount of observations in this dataset is quite small, which means it would be important to use a t-distribution to calculate our confidence interval. 

We can do that with `sm.stats.DescrStatsW(df['INCOME']).tconfint_mean()`. How does using the t-distribution change the confidence interval?

In [ ]:
sm.stats.DescrStatsW(df['INCOME']).tconfint_mean()

Now let's have a look at how to do a t-test. We're going to look at whether income is systematically higher or lower in southern states in the dataset. First, write a little bit of code that works out the difference in means between the two states. To get you started, you could use `df[df['SOUTH']==1]['INCOME'].mean()` to get the mean of all states which have south = 1.   

In [7]:
df[df['SOUTH']==1]['INCOME'].mean() - df[df['SOUTH']==0]['INCOME'].mean()

-3633.3714285714304

Now let's run the t-test. Statsmodels has a library to do this for us: 

`sm.stats.ttest_ind(df[df['SOUTH']==1]['INCOME'], df[df['SOUTH']==0]['INCOME'], usevar="unequal")`. You can see the documentation here: https://www.statsmodels.org/dev/generated/statsmodels.stats.weightstats.ttest_ind.html

This returns three values: the test statistics, the p value, and the degrees of freedom 

In [8]:
sm.stats.ttest_ind(df[df['SOUTH']==1]['INCOME'], df[df['SOUTH']==0]['INCOME'], usevar="unequal")

(-1.587305006418155, 0.134947304616254, 13.885455969119226)

Challenge: write out a function that performs a t test and returns the test statistic. This will involve taking into two arrays of numbers, calculating the difference in means, and expressing that difference in means in terms of a number of standard errors. Check your result agrees with the function above. If you want to do even more, you could also calculate the degrees of freedom (see formula here: https://www.statisticshowto.datasciencecentral.com/satterthwaite-formula/) and hence determine the p value. Note that you can use `sp.stats.t.cdf(ts, df=deg_freedom)*2` to convert a test statistics (`ts`) and degrees of freedom (`deg_freedom`) into a p value. 

In [9]:
def calc_dof(x, y): 
    dof_num = (x.var()/x.shape[0] + y.var()/y.shape[0])**2 
    dof_denom = ((x.var()/x.shape[0])**2 / (x.shape[0]-1) + (y.var()/y.shape[0])**2 / (y.shape[0]-1))
    return(dof_num/dof_denom)

In [10]:
def t_test(df=None, var=None, group=None):
    #check there are two groups
    groups = df[group].unique()
    if len(groups)!=2:print('error - group variable must have exactly two values');return
    
    #calculate group means, variance and n
    means = [df[df[group]==g][var].mean() for g in groups]
    variances = [df[df[group]==g][var].var() for g in groups] 
    ns = [df[df[group]==g][var].shape[0] for g in groups]
    
    se_2 = (variances[0] / ns[0]) + (variances[1] / ns[1])
    se = np.sqrt(se_2)
    
    deg_freedom = calc_dof(*(df[df[group]==g][var] for g in groups))
    t = (means[0] - means[1]) / se
    p = sp.stats.t.cdf(t, df=deg_freedom)*2
    print('Comparing groups:', groups, 'for variable', group)
    print('Mean for group', groups[0], ':', means[0])
    print('Mean for group', groups[1], ':', means[1])
    print('Difference in means:', means[0] - means[1])
    print('Test statistic:', t)
    print('Degrees of freedom:', deg_freedom)
    print('P value:', p)
    

In [11]:
t_test(df=df, var='INCOME', group='SOUTH')

Comparing groups: [1. 0.] for variable SOUTH
Mean for group 1.0 : 33325.42857142857
Mean for group 0.0 : 36958.8
Difference in means: -3633.3714285714304
Test statistic: -1.587305006418155
Degrees of freedom: 13.885455969119223
P value: 0.1349473046162541
